In [ ]:
import json

with open("essential_account.json","r",encoding="UTF-8") as file:
    data = json.load(file)
    accounts = data["essential_accounts"]

for account in accounts:
    print(account)
    account["name"]




In [ ]:
import pandas as pd 

from scripts.player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager
product_A = game_master.construct_instance("inventory","product_A").get("ID")
bilding_B = game_master.construct_instance("building", "building_B", address="東京都品川区荏原3-7-13").get("ID")
asset_list = list(game_master.asset_registry.keys())
player1.redister_product(product_A)
player1.aquire_building(bilding_B, 2000)

player1.purchase_product(product_A, 100, 150)
player1.purchase_product(product_A, 200, 120, 15)

player1.sale_product(product_A, 80)

player1.purchase_product(product_A, 30, 160)
player1.sale_product(product_A, 100, 225)

player1.sale_product(product_A, 100, 210, 500)

player1.perform_inventory_audit(product_A)

end_1 = ledger1.execute_settlement()

ledger1.display_transaction_history()
# ledger1.display_trial_balance(end_1)
# ledger1.display_financial_statements(end_1)

df = pd.DataFrame.from_dict(end_1, orient="index")


In [ ]:
ledger1.display_financial_statements(end_1)

In [ ]:
inventory_c = game_master.construct_instance("inventory", "inventory_c")
type(inventory_c.get("class"))


In [ ]:
import pandas as pd 

from scripts.player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager

building_B = game_master.construct_instance("building")

In [ ]:
from scripts.player import *
 # ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーを作成して登録
player = Player(name="Player1", game_master=game_master)
game_master.players.append(player)

# 資産の生成と登録
tangible_asset = game_master.construct_instance(
    asset_type="building",
    name="OfficeBuilding",
    value=1000000,
    address="Central Tokyo"
)["ID"]

# プレイヤーに資産を取得させる
player.aquire_building(asset_id=tangible_asset, value=1000000)

# ゲーム内時間を90日進める
print("\n=== 時間を90日進めます ===")
game_master.advance_time(days=90)

# 資産の状態を表示
print("\n=== 資産の状態 ===")
for asset_info in player.portfolio:
    asset = game_master.get_asset_by_id(asset_info["ID"])["instance"]
    print(f"資産名: {asset.name}, 帳簿価額: {int(asset.value)}, 累計減価償却額: {int(asset.accumulated_depreciation)}")

# 勘定元帳の記録を確認
print("\n=== プレイヤーの勘定元帳 ===")
player.ledger_manager.display_transaction_history()

In [ ]:
from scripts.asset import *
b = Building(name="Test Building", value=100000, owner=None, address="Test Address")
print(isinstance(b, Tangible))  # True を期待

In [ ]:
import scripts.asset
print(asset.__file__)


In [1]:
from scripts.player import *

# ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーを作成して登録
player = Player(name="Player1", game_master=game_master)
game_master.players.append(player)

# 資産の生成と登録
building_asset = game_master.construct_instance(
    asset_type="building",
    name="Office Building",
    value=1000000,
    address="Downtown City"
)

# プレイヤーに建物資産を取得させる
player.aquire_building(asset_id=building_asset["ID"], value=1000000)

# ゲーム内時間を進める
print("\n=== 90日進行 ===")
game_master.advance_time(days=90)

# 資産の確認
print("\n=== プレイヤーのポートフォリオ ===")
for asset_info in player.portfolio:
    asset = asset_info.get("instance")
    print(f"資産名: {asset.name}, 帳簿価額: {int(asset.value)}, 累計減価償却額: {int(asset.accumulated_depreciation)}")

# 勘定元帳の記録を確認
print("\n=== プレイヤーの勘定元帳 ===")
player.ledger_manager.display_transaction_history()

資産 'Office Building' (ID: 4e475772-f60e-4d71-b98e-c4ece8bb4e6e, クラス: <class 'asset.Building'>) が登録されました。
Office Building の所有者が Player1 に設定されました。
Building instance type: <class 'asset.Building'>

=== 90日進行 ===
イベント記録: {'date': '2024-03-31', 'event': '90日進行', 'details': {}}
Office Building の減価償却が適用されました: 6164 減価償却累計額: 6164 残存価額: 993836
Office Building の時間経過が処理されました (90日)。
90日間時間が進行しました。現在日時: 2024-03-31

=== プレイヤーのポートフォリオ ===
資産名: Office Building, 帳簿価額: 993836, 累計減価償却額: 6164

=== プレイヤーの勘定元帳 ===


取引一覧:

  [ゲーム内時間]
    仕訳: 現金: 5000, 資本金: -5000
    摘要: Initial capital
  [ゲーム内時間]
    仕訳: 建物: 1000000, 現金: -1000000
    摘要: 建物の取得　建物名：Office Building
  [ゲーム内時間]
    仕訳: 減価償却費: 6164, 減価償却累計額: -6164
    摘要: Office Building の減価償却 (90日)
